<a href="https://colab.research.google.com/github/eshika289/cs301_project/blob/main/CS_301_NLP_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

https://jax.readthedocs.io/en/latest/notebooks/quickstart.html

In [ ]:
import os, shutil
import csv
import re
import math
from collections import Counter
from collections import defaultdict

In [ ]:
from typing import List
from typing import Any
from typing import Dict
from typing import TypeVar

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Get Data From Kaggle

Get kaggle.json file from kaggle and upload it to the content folder

In [ ]:
!pwd # /content

/content


In [ ]:
!mkdir /root/.kaggle
!mkdir kaggle

In [ ]:
mv ./kaggle.json /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json

In [ ]:
#!pip install kaggle
!kaggle -v

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions files -c nlp-getting-started

name                    size  creationDate         
---------------------  -----  -------------------  
sample_submission.csv   22KB  2019-12-16 20:36:20  
test.csv               411KB  2019-12-16 20:36:20  
train.csv              965KB  2019-12-16 20:36:20  


In [ ]:
!kaggle competitions download -c nlp-getting-started -p /content/kaggle/

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 90.1MB/s]


In [ ]:
!unzip -q /content/kaggle/nlp-getting-started.zip -d /content/kaggle/data

In [ ]:
data_path = "kaggle/data/train.csv"
data_file = open(data_path)
read_file = csv.reader(data_file)
headers = []
headers = next(read_file)
data = []
for row in read_file:
  data.append(row)

In [ ]:
print(headers) # ['id', 'keyword', 'location', 'text', 'target']
print(data[0])

['id', 'keyword', 'location', 'text', 'target']
['1', '', '', 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', '1']


# Most Common Words as Features

In [ ]:
def most_common_words(subset):
  all_text = ""
  for i in range(len(subset)):
    all_text+=subset[i][3]+" "

  stop_words = stopwords.words('english')
  all_text.lower()
  words_list = all_text.split()
  word_list = []
  part_of_speech = {}

  for word in words_list:
    if word not in stop_words and re.search('[a-z]', word) is not None and word!="&amp;" and word[:4]!="http":
      #tmp = wn.synsets(word)[0].pos()
      word_list.append(word.lower())
      #part_of_speech[word.lower]= tmp

  #words_dict = dict(sorted(words_dict.items(), key=lambda item: -item[1]))
  words_dict = Counter(word_list)
  for item in words_dict.most_common(50):
    print(item)
    #print(part_of_speech[item])

most_common_words(data)

('the', 552)
('like', 335)
("i'm", 229)
('get', 224)
('via', 213)
('fire', 203)
('new', 203)
('one', 178)
('people', 168)
('in', 155)
('emergency', 135)
('police', 125)
('still', 125)
('would', 124)
('disaster', 118)
('body', 116)
('this', 110)
('got', 108)
('to', 108)
('burning', 108)
('video', 107)
('suicide', 107)
('see', 102)
('first', 100)
('back', 100)
('know', 99)
('going', 99)
('man', 96)
('you', 96)
('storm', 96)
('my', 95)
('two', 95)
('killed', 95)
('love', 91)
('time', 91)
('california', 90)
('bomb', 90)
('if', 89)
('nuclear', 89)
('full', 88)
('may', 87)
('buildings', 87)
('go', 86)
('families', 86)
("can't", 85)
('crash', 84)
('train', 84)
('car', 83)
('@youtube', 83)
('day', 83)


# Keywords as Features

In [ ]:
keywords = defaultdict(int)
for keyword in [data[x][1] for x in range(len(data))]:
  keywords[str(keyword)]+=1

keywords = {k: v for k, v in sorted(keywords.items(), key=lambda item: -item[1])}

print(keywords)

{'': 61, 'fatalities': 45, 'armageddon': 42, 'deluge': 42, 'body%20bags': 41, 'damage': 41, 'harm': 41, 'sinking': 41, 'collided': 40, 'evacuate': 40, 'fear': 40, 'outbreak': 40, 'siren': 40, 'twister': 40, 'windstorm': 40, 'collision': 39, 'derailment': 39, 'earthquake': 39, 'explosion': 39, 'famine': 39, 'flames': 39, 'hellfire': 39, 'sinkhole': 39, 'sunk': 39, 'weapon': 39, 'weapons': 39, 'whirlwind': 39, 'wreckage': 39, 'wrecked': 39, 'ambulance': 38, 'blaze': 38, 'bombed': 38, 'deaths': 38, 'derailed': 38, 'drowned': 38, 'explode': 38, 'fatal': 38, 'fire': 38, 'flooding': 38, 'hurricane': 38, 'injury': 38, 'oil%20spill': 38, 'thunder': 38, 'typhoon': 38, 'upheaval': 38, 'bioterror': 37, 'blizzard': 37, 'burning%20buildings': 37, 'crush': 37, 'curfew': 37, 'debris': 37, 'destroy': 37, 'emergency': 37, 'fatality': 37, 'hostages': 37, 'mudslide': 37, 'pandemonium': 37, 'panic': 37, 'police': 37, 'quarantined': 37, 'ruin': 37, 'sandstorm': 37, 'wounded': 37, 'wreck': 37, 'ablaze': 36,

In [ ]:
def by_keyword(word):
  valid = 0
  falseAlarm = 0
  for item in data:
    if item[1]==word:
      if item[4]=='1':
        valid+=1
      else:
        falseAlarm+=1
  print("Valid : " + str(valid))
  print("False : " + str(falseAlarm))

def keyword_subset(word):
  subset = []
  for item in data:
    if item[1]==word:
      subset.append(item)
  return subset

by_keyword("destroy")


Valid : 9
False : 28


## Words that Come Before/After Keywords

# Locations As Features

In [ ]:
def by_location(location): 
  locations_v = []
  locations_f = []
  for item in data:
    if item[4]=="1":
      locations_v.append(item[2])
    else:
      locations_f.append(item[2])
  locations_v = Counter(locations_v)
  locations_f = Counter(locations_f)
  print(str(location) + ": Valid : " + str(locations_v[location]))
  print(str(location) + ": False : " + str(locations_f[location]))

# Bigrams / Trigrams As Features

# Grammars (nouns,verbs) As Features

# Decision Tree

Entropy

In [ ]:
def entropy(class_probabilities: List[float]) -> float:
  '''Given a list of class probabilities, compute the entropy'''
  return sum(-p*math.log(p,2) for p in class_probabilities if p > 0)

def class_probabilities(labels: List[Any]) -> float:
  total_count = len(labels)
  return [count/total_count for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
  return entropy(class_probabilities(labels))

def partition_entropy(subsets: List[List[Any]]) -> float:
  '''Returns the entropy from this partition of data into subsets'''
  total_count = sum(len(subset) for subset in subsets)
  return sum(data_entropy(subset)*len(subset)/total_count for subset in subsets)

Parition

In [ ]:
T = TypeVar('T')

def partition_by(inputs: List[T], attribute:str) -> Dict[Any, List[T]]:
  '''Partitions the inputs into lists based on the specified attribute'''
  partitions: Dict[Any,List[T]] = defaultdict(list)
  for input in inputs:
    key = getattr(input, attribute)
    partitions[key].append(input)
  return partitions

def partition_entropy_by(inputs: List[Any], attribute:str,label_attribute:str) ->float:
  '''Computes the entropy corresponding to the given partition'''
  partitions = partition_by(inputs,attribute)
  labels = [[getattr(input, label_attribute) for input in partition] for partition in partitions.values()]
  return partition_entropy(labels)

Build Tree

# Random Forests